# 2. Heatmap generation

Notebook used to generate ground-truth for the probability maps. 

In [ ]:
import numpy as np 
import pickle
import matplotlib.pyplot as plt
import cv2

Function to generate heatmap for each image:

In [ ]:
def get_pdf(im, kpts, sigma):
    w, h, channels = im.shape
    
    x = np.linspace(0, h-1, h*1)
    y = np.linspace(0, w-1, w*1)
    [XX, YY] =  np.meshgrid(y,x)
    sze = XX.shape[0] * XX.shape[1]
    mvg = np.zeros((sze));    
    std = sigma
    p = 2
    count=0
    for i in range(0,37):
        mu = np.array([kpts[i][1], kpts[i][0]]).reshape((2,1)) 
        mu = np.tile(mu, (1, sze))
        mcov = np.identity(2) * std
        
        X = np.array([np.ravel(XX.T), np.ravel(YY.T)])
        
        temp0 = 1 / ( math.pow(2*math.pi, p/2) * \
                    math.pow(np.linalg.det(mcov), 0.5) )
        
        temp1 = -0.5*(X-mu).T
        temp2 = np.linalg.inv(mcov).dot(X-mu) 
        
        temp3 = temp0 * np.exp(np.sum(temp1 * temp2.T, axis=1))
        maximum = max(temp3.ravel())
        
        mvg = mvg + temp3
        count += 1
    
        mvg[mvg>maximum] = maximum
        
    mvg = mvg.reshape((XX.shape[1], XX.shape[0]))
    
    mvg = ( mvg - min(mvg.ravel()) ) / ( max(mvg.ravel()) - min(mvg.ravel()) )
    
    mvg = mvg * 255.0
    mvg = cv2.resize(mvg, (h, w), interpolation = cv2.INTER_CUBIC)
    mvg = mvg / 255.0
    mvg[mvg<0] = 0
   
    return mvg

To feed the previous function, a tupple of keypoints (x,y) is needed. The next function transforms array of keypoints in array of tupples:

In [ ]:
def tupple(data):
    
    points = np.array(data)
    
    tupple_keypoints = []
    tupple_aux = [] 
    x = []
    y = []
    
    for i in range(points.shape[0]): 
        for j in range(74): 
            if (j % 2 == 0): 
                x.append(int(points[i][j]))
            else:
                y.append(int(points[i][j]))    
        for z in range(37):
            tupple_aux.append((int(x[z]),int(y[z])))
        tupple_keypoints.append(tupple_aux)
        x = [] 
        y = [] 
        tupple_aux = [] 
    
    for i in range(points.shape[0]):
        tupple_keypoints.append(points[i])
    
    keypoints = np.array(tupple_keypoints)

    return keypoints

Function to generate heatmaps for dataset:

In [ ]:
def heatmap_generation(X, keypoints):
    density_map = []
    for i in range(len(X)):
        oriImg = X[i]
        mapa = get_pdf(oriImg,keypoints[i],400)
        density_map.append(mapa)
    return np.array(density_map)

Read pre-processed data (images and keypoints):

In [ ]:
with open('pre_processed_data/X_train_preprocessed.pickle', 'rb') as f:
    X_train = pickle.load(f)

with open('pre_processed_data/y_train_preprocessed.pickle', 'rb') as f:
    data_train = pickle.load(f)

Apply functions created:

In [ ]:
keypoints_train = tupple(data_train)    

density_map_train = heatmap_generation(X_train, keypoints_train)

Save ground-truth:

In [ ]:
with open('pre_processed_data/heatmap_train.pickle', 'wb') as f:
    pickle.dump(density_map_train, f)